# ablation study
How accurate can we make a recurring timer in a linux server?

In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# compile userspace timer test
!cargo run -- -s 10000 -a 100000 -r 0

    Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`dev` profile [unoptimized + debuginfo]]8;;\ target(s) in 0.01s
     Running `target/debug/sleep -s 10000 -a 100000 -r 0`
Desired sleep duration of 10000 nanoseconds: 100000 runs
Avg: 10.099µs
Std: 332ns
Max: 39.677µs
Min: 10.052µs
jitter: 29.625us


In [ ]:
# stress test cpus
!stress-ng -l 50 -c 4 

stress-ng: info:  [41313] defaulting to a 1 day run per stressor
stress-ng: info:  [41313] dispatching hogs: 4 cpu
stress-ng: info:  [41314] cpu: for stable load results, select a specific cpu stress method with --cpu-method other than 'all'
stress-ng: info:  [41315] cpu: for stable load results, select a specific cpu stress method with --cpu-method other than 'all'
stress-ng: info:  [41316] cpu: for stable load results, select a specific cpu stress method with --cpu-method other than 'all'
stress-ng: info:  [41317] cpu: for stable load results, select a specific cpu stress method with --cpu-method other than 'all'


In [ ]:
!pkill stress-ng